In [ ]:
# importa todo y define una función de ejecución de pruebas
from importlib import reload
from ayudante import ejecutar_prueba

import bloque
import cce
import ayudante
import bloquemerkle
import network
import script
import tx

### Ejercicio 1

#### Comprueba en la chuleta la estructura del mensaje de red.

#### 1.1. Parsea este mensaje
```
f9beb4d976657261636b000000000000000000005df6e0e2
```

#### 1.2. Haz que pasen [estas pruebas](/edit/sesión7/network.py)
```
network.py:PruebaNetwork:prueba_parsear
network.py:PruebaNetwork:prueba_serializar
```

In [ ]:
# Ejercicio 1.1

msj = bytes.fromhex('f9beb4d976657261636b000000000000000000005df6e0e2')

# los primeros 4 son la magia de red
# los siguientes 12 son el comando
# los siguientes 4 son la longitud de la carga
# los siguientes 4 son el checksum
# el resto es la carga

In [ ]:
# Ejercicio 1.2

reload(network)
ejecutar_prueba(network.PruebaNetworkEnvelope('prueba_parsear'))
ejecutar_prueba(network.PruebaNetworkEnvelope('prueba_serializar'))

### Ejercicio 2

#### 2.1. Haz que pasen [esta prueba](/edit/sesión7/network.py)
```
network.py:PruebaVersiónMensaje:prueba_serializar
```

In [ ]:
# Ejercicio 2.1

reload(network)
ejecutar_prueba(network.PruebaVersiónMensaje('prueba_serializar'))

### Ejercicio 3

#### 3.1. Haz que pase [esta prueba](/edit/sesión7/network.py)
```
network.py:PruebaObtenerMensajeCabeceras:prueba_serializar
```

#### 3.2. Haz que pase [esta prueba](/edit/sesión7/network.py)
```
network.py:PruebaMensajeCabeceras:prueba_parsear
```

In [ ]:
# Ejercicio 3.1

reload(network)
ejecutar_prueba(network.PruebaObtenerMensajeCabeceras('prueba_serializar'))

In [ ]:
# Ejercicio 3.2

reload(network)
ejecutar_prueba(network.PruebaMensajeCabeceras('prueba_parsear'))

In [ ]:
# Ejemplo Handshake

from network import NodoSimple, MensajeVersión

node = NodoSimple('tbtc.programmingbloquechain.com', testnet=True, logging=True)

versión = MensajeVersión()
node.send(versión.command, versión.serializar())
print(node.esperar_comandos([b'verack']))

### Ejercicio 4

#### 4.1. Haz que pase [esta prueba](/edit/sesión7/network.py)
```
network.py:PruebaNodoSimple:prueba_handshake
```

In [ ]:
# Ejercicio 4.1

reload(network)
ejecutar_prueba(network.PruebaNodoSimple('prueba_handshake'))

In [ ]:
# Ejemplo de descarga de cabecera de bloque

from bloque import HASH_BLOQUE_GENESIS
from network import ObtenerCabeceraMensaje, CabeceraMensaje, NodoSimple

node = NodoSimple('btc.programmingbloquechain.com', testnet=False)
node.handshake()
último_hash_bloque = HASH_BLOQUE_GENESIS
contador = 1
for _ in range(20):
    obtenercabeceras = ObtenerCabeceraMensaje(bloque_inicial=último_hash_bloque)
    node.send(obtenercabeceras.command, obtenercabeceras.serializar())
    envelope_cabeceras = node.esperar_comandos([b'headers'])
    mensaje_cabeceras = CabeceraMensaje.parsear(envelope_cabeceras.stream())
    for b in mensaje_cabeceras.bloques:
        if not b.comprobar_pow():
            raise RuntimeError('prueba de trabajo mala en el bloque {}'.format(contador))
        if último_hash_bloque != HASH_BLOQUE_GENESIS and b.bloque_previo != último_hash_bloque:
            raise RuntimeError('bloque discontinuo en {}'.format(contador))
        contador += 1
        último_hash_bloque = b.hash()
        if contador % 2016 == 0:
            print(b.hash().hex())

### Ejercicio 5

#### 5.1. Descarga los primeros 40,000 bloques para testnet y valídalos.


In [ ]:
# Ejercicio 5.1

from bloque import HASH_BLOQUE_GÉNESIS_TESTNET
from network import ObtenerCabeceraMensaje, CabeceraMensaje, NodoSimple

# conéctate a tbtc.programmingbloquechain.com
# handshake
# establece el hash del último bloque como HASH_BLOQUE_GÉNESIS_TESTNET
# establece el contador de bloques a 1
# haz un bucle hasta que tengamos 40,000 bloques
    # crea un ObtenerCabeceraMensaje empezando por el último bloque que tengamos
    # envía el comando obtenercabeceras
    # espera el comando de cabeceras como respuesta
    # parsea el network envelope que se devuelve
    # haz un bucle a través de los bloques desde los mensajes cabecera
        # comprueba la prueba de trabajo
        # el bloque_previo del bloque actual debería ser el último bloque
        # aumenta el contador del bloque
        # establece el hash del último bloque
        # imprime la cabecera del bloque cada 2016 bloques (ajuste de la dificultad)

In [ ]:
# Ejemplo de Padre Merkle

from ayudante import doble_sha256

tx_hash0 = bytes.fromhex('c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5')
tx_hash1 = bytes.fromhex('c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5')

padre_merkle = doble_sha256(tx_hash0+tx_hash1)
print(padre_merkle.hex())

### Ejercicio 6

#### 6.1. Calcula el padre merkle de estos hashes:
```
f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0
3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181
```

#### 6.2. Haz que pase [esta prueba](/edit/sesión7/ayudante.py)
```
ayudante.py:PruebaAyudante:prueba_padre_merkle
```

In [ ]:
# Ejercicio 6.1

from ayudante import doble_sha256

hex_hash1 = 'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0'
hex_hash2 = '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181'

# bytes.fromhex para obtener los hashes binarios
# doble_sha256 la combinación
# hex() para ver el resultado

In [ ]:
# Ejercicio 6.2

reload(ayudante)
ejecutar_prueba(ayudante.PruebaAyudante('prueba_padre_merkle'))

In [ ]:
# Ejemplo de Nivel de padre Merkle

from ayudante import doble_sha256, padre_merkle
hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
]
hashes = [bytes.fromhex(x) for x in hex_hashes]
if len(hashes) % 2 == 1:
    hashes.append(hashes[-1])
nivel_padre = []
for i in range(0, len(hex_hashes), 2):
    padre = padre_merkle(hashes[i], hashes[i+1])
    print(padre.hex())
    nivel_padre.append(padre)

### Ejercicio 7

#### 7.1. Calcula el siguiente nivel de padre merkle dados estos hashes
```
8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd
7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800
ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7
68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069
43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27
```

#### 7.2. Haz que pase [esta prueba](/edit/sesión7/ayudante.py)
```
ayudante.py:PruebaAyudante:prueba_nivel_padre_merkle
```

In [ ]:
# Ejercicio 7.1

from ayudante import padre_merkle

hex_hashes = [
    '8b30c5ba100f6f2e5ad1e2a742e5020491240f8eb514fe97c713c31718ad7ecd',
    '7f4e6f9e224e20fda0ae4c44114237f97cd35aca38d83081c9bfd41feb907800',
    'ade48f2bbb57318cc79f3a8678febaa827599c509dce5940602e54c7733332e7',
    '68b3e2ab8182dfd646f13fdf01c335cf32476482d963f5cd94e934e6b3401069',
    '43e7274e77fbe8e5a42a8fb58f7decdb04d521f319f332d88e6b06f8e6c09e27',
]

# bytes.fromhex para obtener todos los hashes en binario
# si el número de hashes es impar, dsubirlica el último
# inicializa el nivel padre
# salta de dos en dos: usa range(0, len(hashes), 2)
    # calcula padre_merkle de i y i+1 hashes
    # imprime el hex del hash
    # añade padre al nivel padre

In [ ]:
# Ejercicio 7.2

reload(ayudante)
ejecutar_prueba(ayudante.PruebaAyudante('prueba_nivel_padre_merkle'))

In [ ]:
# Ejemplo de Raiz Merkle

from ayudante import doble_sha256, nivel_padre_merkle

hex_hashes = [
    'c117ea8ec828342f4dfb0ad6bd140e03a50720ece40169ee38bdc15d9eb64cf5',
    'c131474164b412e3406696da1ee20ab0fc9bf41c8f05fa8ceea7a08d672d7cc5',
    'f391da6ecfeed1814efae39e7fcb3838ae0b02c02ae7d0a5848a66947c0727b0',
    '3d238a92a94532b946c90e19c49351c763696cff3db400485b813aecb8a13181',
    '10092f2633be5f3ce349bf9ddbde36caa3dd10dfa0ec8106bce23acbff637dae',
    '7d37b3d54fa6a64869084bfd2e831309118b9e833610e6228adacdbd1b4ba161',
    '8118a77e542892fe15ae3fc771a4abfd2f5d5d5997544c3487ac36b5c85170fc',
    'dff6879848c2c9b62fe652720b8df5272093acfaa45a43cdb3696fe2466a3877',
    'b825c0745f46ac58f7d3759e6dc535a1fec7820377f24d4c2c6ad2cc55c0cb59',
    '95513952a04bd8992721e9b7e2937f1c04ba31e0469fbe615a78197f68f52b7c',
    '2e6d722e5e4dbdf2447ddcce9f7dabb8e299bae921c99ad5b0184cd9eb8e5908',
    'b13a750047bc0bdceb2473e5fe488c2596d7a7124b4e716fdd29b046ef99bbf0',
]

nivel_actual = [bytes.fromhex(x) for x in hex_hashes]
while len(nivel_actual) > 1:
    nivel_actual = nivel_padre_merkle(nivel_actual)
print(nivel_actual[0].hex())

### Ejercicio 8

#### 8.1. Calcula la raíz merkle dados estos hashes
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```

#### 8.2. Haz que pase [esta prueba](/edit/sesión7/ayudante.py)
```
ayudante.py:PruebaAyudante:prueba_raiz_merkle
```

In [ ]:
# Ejercicio 8.1

from ayudante import doble_sha256, nivel_padre_merkle
hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]

# bytes.fromhex para obtener todos los hashes en binario
# inicializa el nivel actual para ser los hashes
# haz un bucle hasta que nivel_actual tenga 1 solo elemento
    # haz que el nivel actual sea el nivel padre
# imprime el hex de la raíz

In [ ]:
# Ejercicio 8.2

reload(ayudante)
ejecutar_prueba(ayudante.PruebaAyudante('prueba_raiz_merkle'))

In [ ]:
# Ejemplo Raiz Bloque Merkle

from ayudante import doble_sha256, nivel_padre_merkle, raiz_merkle
tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
]
nivel_actual = [bytes.fromhex(x)[::-1] for x in tx_hex_hashes]
print(raiz_merkle(nivel_actual)[::-1].hex())

### Ejercicio 9

#### 9.1. Valida la raíz merkle de este bloque en Testnet:
Hash del Bloque:
```
0000000000000451fa80fcdb243b84c35eaae215a85a8faa880559e8239e6f20
```

Hashes de la transacción:
```
42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e
94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4
959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953
a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2
62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577
766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba
e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208
921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e
15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321
1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0
3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d
```

#### 9.2. Haz que pase [esta prueba](/edit/sesión7/bloque.py)
```
bloque.py:PruebaBloque:prueba_validar_raiz_merkle
```

In [ ]:
# Ejercicio 9.1

from ayudante import doble_sha256, raiz_merkle

want = bytes.fromhex('4297fb95a0168b959d1469410c7527da5d6243d99699e7d041b7f3916ba93301')

tx_hex_hashes = [
    '42f6f52f17620653dcc909e58bb352e0bd4bd1381e2955d19c00959a22122b2e',
    '94c3af34b9667bf787e1c6a0a009201589755d01d02fe2877cc69b929d2418d4',
    '959428d7c48113cb9149d0566bde3d46e98cf028053c522b8fa8f735241aa953',
    'a9f27b99d5d108dede755710d4a1ffa2c74af70b4ca71726fa57d68454e609a2',
    '62af110031e29de1efcad103b3ad4bec7bdcf6cb9c9f4afdd586981795516577',
    '766900590ece194667e9da2984018057512887110bf54fe0aa800157aec796ba',
    'e8270fb475763bc8d855cfe45ed98060988c1bdcad2ffc8364f783c98999a208',
    '921b8cfd3e14bf41f028f0a3aa88c813d5039a2b1bceb12208535b0b43a5d09e',
    '15535864799652347cec66cba473f6d8291541238e58b2e03b046bc53cfe1321',
    '1c8af7c502971e67096456eac9cd5407aacf62190fc54188995666a30faf99f0',
    '3311f8acc57e8a3e9b68e2945fb4f53c07b0fa4668a7e5cda6255c21558c774d',
]

# bytes.fromhex y dale la vuelta ([::-1]) para obtener todos los hashes en binario
# obtén la raíz de merkle
# comprueba si el reverso de la raíz es el mismo que la raíz deseada

In [ ]:
# Ejercicio 9.2

reload(bloque)
ejecutar_prueba(bloque.PruebaBloque('prueba_validar_raiz_merkle'))

### Ejercicio 10

#### 10.1. Valida la raíz merkle para este bloque en Testnet via el protocolo de red:
Hash del Bloque:
```
0000000000044b01a9440b34f582fe171c7b8642fedd0ebfccf8fdf6a1810900
```

In [ ]:
# Ejercicio 10.1

from bloque import HASH_BLOQUE_GÉNESIS_TESTNET, Bloque
from ayudante import read_varint
from network import TIPO_DATOS_BLOQUE, ObtenerMensajeDatos, NodoSimple
from tx import Tx

bloque_hex = '0000000000044b01a9440b34f582fe171c7b8642fedd0ebfccf8fdf6a1810900'
bloque_hash = bytes.fromhex(bloque_hex)

# conéctate a tbtc.programmingbloquechain.com en testnet
# handshake

# crea un ObtenerMensajeDatos
# añade el bloque que queramos
# envía el comando getdata

# espera el bloque comando como respuesta
# obtén el stream desde el envelope
# el mensaje del bloque es la cabecera del bloque primero, así que parsea el Bloque
# comprueba la prueba de trabajo
# el siguiente mensaje del bloque tiene el número de transacciones así que haz read_varint de eso
# inicializa la lista de tx_hashes
# haz un bucle a través de eso ese número de veces
    # parse la transacción
    # añade el hash de la transacción a la lista de tx_hashes

# establece los tx_hashes del bloque a los que hemos reunido
# valida los tx_hashes para la raíz de merkle
# imprime el hex de la raíz de merkle

In [ ]:
# Ejemplo de Árbol Merkle

import math

total = 16

profundidad_máxima = math.ceil(math.log(total, 2))
árbol_merkle = []
for profundidad in range(profundidad_máxima + 1):
    num_cosas = math.ceil(total / 2**(profundidad_máxima - profundidad))
    nivel_hashes = [None] * num_cosas
    árbol_merkle.append(nivel_hashes)

for level in árbol_merkle:
    print(level)

In [ ]:
# Ejemplo de poblar y navegar un árbol de merkle

from bloquemerkle import ÁrbolMerkle
from ayudante import nivel_padre_merkle, padre_merkle

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
    "f9dbfafc3af3400954975da24eb325e326960a25b87fffe23eef3e7ed2fb610e",
]
árbol = ÁrbolMerkle(len(hex_hashes))
árbol.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
árbol.nodes[3] = nivel_padre_merkle(árbol.nodes[4])
árbol.nodes[2] = nivel_padre_merkle(árbol.nodes[3])
árbol.nodes[1] = nivel_padre_merkle(árbol.nodes[2])
árbol.nodes[0] = nivel_padre_merkle(árbol.nodes[1])
print(árbol)

In [ ]:
# Ejemplo #2 de poblar un Árbol de Merkle

from bloquemerkle import ÁrbolMerkle
from ayudante import padre_merkle

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
    "f9dbfafc3af3400954975da24eb325e326960a25b87fffe23eef3e7ed2fb610e",
]
árbol = ÁrbolMerkle(len(hex_hashes))
árbol.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
while árbol.raíz() is None:
    if árbol.es_hoja():
        árbol.subir()
    else:
        hash_izquierdo = árbol.obtener_nodo_izquierdo()
        hash_derecho = árbol.obtener_nodo_derecho()
        if hash_izquierdo is None:
            árbol.izquierda()
        elif hash_derecho is None:
            árbol.derecha()
        else:
            árbol.establecer_nodo_actual(padre_merkle(hash_izquierdo, hash_derecho))
            árbol.subir()
print(árbol)

In [ ]:
# Ejemplo #3 de poblar Árbol de Merkle
reload(bloquemerkle)
from bloquemerkle import ÁrbolMerkle
from ayudante import padre_merkle

hex_hashes = [
    "9745f7173ef14ee4155722d1cbf13304339fd00d900b759c6f9d58579b5765fb",
    "5573c8ede34936c29cdfdfe743f7f5fdfbd4f54ba0705259e62f39917065cb9b",
    "82a02ecbb6623b4274dfcab82b336dc017a27136e08521091e443e62582e8f05",
    "507ccae5ed9b340363a0e6d765af148be9cb1c8766ccc922f83e4ae681658308",
    "a7a4aec28e7162e1e9ef33dfa30f0bc0526e6cf4b11a576f6c5de58593898330",
    "bb6267664bd833fd9fc82582853ab144fece26b7a8a5bf328f8a059445b59add",
    "ea6d7ac1ee77fbacee58fc717b990c4fcccf1b19af43103c090f601677fd8836",
    "457743861de496c429912558a106b810b0507975a49773228aa788df40730d41",
    "7688029288efc9e9a0011c960a6ed9e5466581abf3e3a6c26ee317461add619a",
    "b1ae7f15836cb2286cdd4e2c37bf9bb7da0a2846d06867a429f654b2e7f383c9",
    "9b74f89fa3f93e71ff2c241f32945d877281a6a50a6bf94adac002980aafe5ab",
    "b3a92b5b255019bdaf754875633c2de9fec2ab03e6b8ce669d07cb5b18804638",
    "b5c0b915312b9bdaedd2b86aa2d0f8feffc73a2d37668fd9010179261e25e263",
    "c9d52c5cb1e557b92c84c52e7c4bfbce859408bedffc8a5560fd6e35e10b8800",
    "c555bc5fc3bc096df0a0c9532f07640bfb76bfe4fc1ace214b8b228a1297a4c2",
]
árbol = ÁrbolMerkle(len(hex_hashes))
árbol.nodes[4] = [bytes.fromhex(h) for h in hex_hashes]
while árbol.raíz() is None:
    if árbol.es_hoja():
        árbol.subir()
    else:
        hash_izquierdo = árbol.obtener_nodo_izquierdo()
        if hash_izquierdo is None:
            árbol.izquierda()
        elif árbol.derecha_exists():
            hash_derecho = árbol.obtener_nodo_derecho()
            if hash_derecho is None:
                árbol.derecha()
            else:
                árbol.establecer_nodo_actual(padre_merkle(hash_izquierdo, hash_derecho))
                árbol.subir()
        else:
            árbol.establecer_nodo_actual(padre_merkle(hash_izquierdo, hash_izquierdo))
            árbol.subir()
print(árbol)